In [2]:
import numpy as np
import torch
import random

In [3]:
target = torch.tensor([[0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.]])
print(target.shape)
target

torch.Size([5, 5])


tensor([[0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [1., 1., 0., 0., 1.]])

In [32]:
target = torch.rand(5,5)
print(target)
target.shape

tensor([[0.9648, 0.6187, 0.7527, 0.3651, 0.8474],
        [0.3475, 0.3877, 0.8914, 0.9127, 0.9432],
        [0.5487, 0.6524, 0.2454, 0.8663, 0.4142],
        [0.5647, 0.6118, 0.7968, 0.0154, 0.4318],
        [0.7454, 0.9327, 0.2163, 0.2600, 0.9836]])


torch.Size([5, 5])

In [40]:
o = torch.tensor([[[[0.3755, 0.0455, 0.0532, 0.8096, 0.0012],
                        [0.8413, 0.6249, 0.7906, 0.3437, 0.7179],
                        [0.6032, 0.0878, 0.8019, 0.3762, 0.4233],
                        [0.8584, 0.8106, 0.0950, 0.4664, 0.9527],
                        [0.2775, 0.6072, 0.2494, 0.4346, 0.3095]]],

                      [[[0.2977, 0.4399, 0.4750, 0.8097, 0.7573],
                        [0.1771, 0.2021, 0.4243, 0.5265, 0.2079],
                        [0.9319, 0.9046, 0.2385, 0.4292, 0.1066],
                        [0.5421, 0.8858, 0.3479, 0.0413, 0.2431],
                        [0.2422, 0.2530, 0.5502, 0.7505, 0.4435]]],

                      [[[0.8230, 0.9267, 0.9465, 0.9285, 0.5393],
                        [0.3142, 0.2686, 0.0248, 0.8731, 0.8203],
                        [0.4774, 0.1330, 0.5620, 0.5715, 0.7523],
                        [0.9411, 0.5292, 0.0644, 0.1066, 0.1095],
                        [0.3790, 0.8215, 0.2276, 0.6534, 0.8838]]]])

In [56]:
output = o[0]
print(output.shape)
output

torch.Size([1, 5, 5])


tensor([[[0.3755, 0.0455, 0.0532, 0.8096, 0.0012],
         [0.8413, 0.6249, 0.7906, 0.3437, 0.7179],
         [0.6032, 0.0878, 0.8019, 0.3762, 0.4233],
         [0.8584, 0.8106, 0.0950, 0.4664, 0.9527],
         [0.2775, 0.6072, 0.2494, 0.4346, 0.3095]]])

In [91]:
a = [[0,2],[3,0]]
b = np.array(a).T
c = np.array([[0,4],[2,3]]).T

In [102]:
anchor = output[0][b]
positive = output[0][c]
print(anchor)
print(positive)

tensor([0.0532, 0.8584])
tensor([0.0012, 0.3762])


In [105]:
torch.cdist(anchor.unsqueeze(0), positive.unsqueeze(0), p=2.0)

tensor([[0.4850]])

In [104]:
anchor - positive

tensor([0.0520, 0.4822])